# TRAIN MODELS
SEND JOB VERTEX

Train differents models. Develop codes to train differents models (in this example the user can't select which models to train. So, all the models can be used)

**It is necesary save in vertex experiments the result of the model trained because it is necesary to access to this information in the future**

**Also it is necesary run the training of the experiments in a vertex jobs (so all of this code needs to be run in a script, with args necesary to identify the case)**

-------
Source codes tracking training: 

folder: https://github.com/joseortegalabra/tracking-training-ml-models/tree/main/vertex_ai

notebook: https://github.com/joseortegalabra/tracking-training-ml-models/blob/main/vertex_ai/3_Vertex_AI_experiments_example.ipynb


------
**Source codes jobs vertex**

repo: https://github.com/joseortegalabra/Jobs-Vertex-GCP

folder example: https://github.com/joseortegalabra/Jobs-Vertex-GCP/tree/main/jobs_vertex_basic_v2

In [ ]:
import datetime as dt
import pandas as pd
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip

### Paso 0. Parámetros generales

In [ ]:
### PARÁMETROS GENERALES GCP - donde está ubicada la app ###
# ---------------------------- read env variables used in the app ----------------------------
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")
REGION_GCP = os.environ.get("REGION_GCP", "")
BUCKET_GCP = os.environ.get("BUCKET_GCP", "")
MAI_SA = os.environ.get("MAIL_SA", "")

In [ ]:
#### DEFINE NAME OF DATASET OF DO THE TRAINING JOB
NAME_DATASET = 'develop-app-final-v2'

In [ ]:
# definir un bucket (ya creado) para guardar los archivos que genera el usar VERTEX AI.
BUCKET_ID = f'{BUCKET_GCP}/vertex-ai-jobs'

In [ ]:
### PARÁMETROS GENERALES EJECUCIÓN ###

# obtener la hora actual de cuándo se comenzó la ejecución - hash
now = dt.datetime.now()
date_time = now.strftime("%Y_%m_%d_%H_%M_%S")

# identificacion del tipo de caso de uso (y también tipo de modelo) que se va a usar poara registrar el entrenamiento
identity_kind_use_case = 'auto-data-science-job'  

# definir path donde se va a guardar el pkl con el modelo y que además quede registrado en modelos de vertex
# definir path CUSTOM donde estará guardado el artefacto del modelo y el cual quedará registrado en el menu "modelos" de vertex
# obligatoriamente el path debe ser ".../model/model.pkl" por lo que la carpeta model y el artefacto model.pkl se omiten en este path
path_artifact_model_vertex = f'gs://{BUCKET_GCP}/vertex-ai-registry-model/{NAME_DATASET}/run_{date_time}/'

In [ ]:
print('Parámetros Generales GCP')
print('PROJECT_GCP: ', PROJECT_GCP)
print('BUCKET_ID: ', BUCKET_ID)
print('REGION_GCP: ', REGION_GCP)

print('\n----')
print('NAME DATASET - USE CASE - EXPERIMENT: ', NAME_DATASET)

print('\n----')
print('Parámetros Específicos job entrenamiento')
print('date_time: ', date_time)
print('identity_kind_use_case: ', identity_kind_use_case)
print('path_artifact_model_vertex: ', path_artifact_model_vertex)

### Paso 1. Crear script de entrenamiento

In [ ]:
# ESCRIBIR EL SCRIPT DE ENTRENAMIENTO.

In [ ]:
# Correr script de entrenamiento de forma local - solo verificar
# %run train_model.py --id_date_time 2023_10_28_18_47_33

### Paso 2: Inicializar Vertex AI

In [ ]:
aiplatform.init(project = PROJECT_GCP, location = REGION_GCP, staging_bucket = BUCKET_ID)

### Paso 3. Definir parámetros necesarios para CREAR la instancia del job de entrenamiento

In [ ]:
### definir el nombre del job que se enviará. Algo que indentifique de qué es el job + hora envio ###
job_name = identity_kind_use_case + '__job_train__' + date_time
job_name

In [ ]:
### definir el contrainer para el ENTRENAMIENTO y para LA PREDICCIÓN - facilitados por google ####
#container_train = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest' 

container_train = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest'
container_deploy = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest' # solo utilizado para servicios de deploy y predicción batch de vertex

In [ ]:
### definir el path al script de entrenamiento ###
path_train_script = 'task_train_models.py'

In [ ]:
### definir la descripción del modelo ###
description = 'entrenar modelo leyendo pkl de GCS'

In [ ]:
### definir los requirements ###
#list_requirements = ["google-cloud-bigquery==3.11.4", "db-dtypes", "gcsfs==2023.9.2", "pandas==2.0.3", "numpy==1.23.5", "scikit-learn==1.3.1"]
list_requirements = ["pandas", "matplotlib", "seaborn", "plotly", "numpy", "scikit-learn", "python-dotenv", "gcsfs", \
                     "joblib", "openpyxl", "google-cloud-bigquery", "db-dtypes", "google-cloud-aiplatform"]

### Paso 4. Definir parámetros necesarios para ENVIAR job de entrenamiento - usando CPU

In [ ]:
### definir el nombre con el que queda registrado (en VERTEX AI) el modelo resultado del entrenamiento ###
# De qué es el modelo +  hora de envio
model_name = identity_kind_use_case  + '__model__' + date_time 
model_name

In [ ]:
### definir el tipo de máquina para hacer el entrenamiento ###

machine_type_train = "n1-standard"
vcpu_train = "4"
train_compute = machine_type_train + "-" + vcpu_train

print("Train machine type: ", train_compute)

### Paso 5. Crear instancia del job de entrenamiento a VERTEX AI (CustomTrainingJob)

In [ ]:
# PRIMERO SE LLAMA UNA INSTANCIA DE LA CLASE
job = aiplatform.CustomTrainingJob(
    display_name = job_name,
    script_path = path_train_script,
    model_description = description,
    container_uri = container_train,
    requirements = list_requirements,
    model_serving_container_image_uri = container_deploy,
)

In [ ]:
job

### Paso 6. Enviar el job de entrenamiento a VERTEX AI (CustomTrainingJob)
- Importante 4. Para que **no aparesca el texto que está corriendo el job y se pueda seguir utilizando el código se puede utilizar el parámetro "sync" y setear en "False"**

In [ ]:
# # original

# model = job.run(
#     model_display_name = model_name,
#     replica_count = 1,
#     machine_type = train_compute,
#     base_output_dir = path_artifact_model_vertex, # path custom .../model/model.pkl donde se guarda el pkl del modelo. se omite del path model/model.pkl
#     args = ["--id_date_time=" + date_time, "--name_dataset=" + NAME_DATASET], # args que se le pasan al script de entrenamiento de este ejemplo
#     sync = True
# )

In [ ]:
# add SA
model = job.run(
    model_display_name = model_name,
    replica_count = 1,
    machine_type = train_compute,
    base_output_dir = path_artifact_model_vertex, # path custom .../model/model.pkl donde se guarda el pkl del modelo. se omite del path model/model.pkl
    args = ["--id_date_time=" + date_time, "--name_dataset=" + NAME_DATASET], # args que se le pasan al script de entrenamiento de este ejemplo
    sync = True,
    service_account = f"{MAI_SA}@{PROJECT_GCP}.iam.gserviceaccount.com"
)